---
# Object Detection Project

## Importing Essential Librarie

In [1]:
import numpy as np
import imutils
import cv2

## Loading MobileNetSSD Model with CUDA Acceleration

In [2]:
# Load the model
net = cv2.dnn.readNetFromCaffe('MobileNetSSD_model.prototxt', 'MobileNetSSD_model.caffemodel')

# Set preferable backend and target to CUDA if using GPU
use_gpu = True
if use_gpu:
    print("[INFO] setting preferable backend and target to CUDA...")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

[INFO] setting preferable backend and target to CUDA...


## Defining Classes, Colors, and Confidence Threshold for Object Detection

In [3]:
# Load classes and colors
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# Confidence threshold
confidence_level = 0.5

## Real-time Object Detection with MobileNetSSD Model and Webcam

In [ ]:
# Initialize the video capture (webcam)
vs = cv2.VideoCapture(0)

while True:
    ret, frame = vs.read()
    if not ret:
        break

    frame = cv2.resize(frame, (600, 600))
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    (h, w) = frame.shape[:2]
    
    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > confidence_level:
            idx = int(detections[0, 0, i, 1])
            
            if idx >= 0 and idx < len(CLASSES):
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
                color = COLORS[idx]
                
                # Draw bounding box and label
                cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            else:
                print("Invalid class index:", idx)

    cv2.imshow('Live detection', frame)
    if cv2.waitKey(1) == 27:
        break

# Release the video capture and close the window
vs.release()
cv2.destroyAllWindows()
